In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import recall_score, precision_score

In [2]:
df = pd.read_csv('Data/diabetic_data.csv')
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


### Background on the dataset

This dataset comes from a study published in BioMed Research International that investigated the effects of HbA1c measurement on hospital readmission rates in the US. Fortunately for me, the dataset has a thorough description of the variable types and % missing. The original study will be included in the GitHub repo for reference.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
encounter_id                101766 non-null int64
patient_nbr                 101766 non-null int64
race                        101766 non-null object
gender                      101766 non-null object
age                         101766 non-null object
weight                      101766 non-null object
admission_type_id           101766 non-null int64
discharge_disposition_id    101766 non-null int64
admission_source_id         101766 non-null int64
time_in_hospital            101766 non-null int64
payer_code                  101766 non-null object
medical_specialty           101766 non-null object
num_lab_procedures          101766 non-null int64
num_procedures              101766 non-null int64
num_medications             101766 non-null int64
number_outpatient           101766 non-null int64
number_emergency            101766 non-null int64
number_inpatient            10176

### Data cleaning

None of the data is null, but many values are missing in certain variables according to the description. We will need to handle these features before modelling the data. Additionally, we will engineer some new features.

In [58]:
# use copy of df to preserve original data, drop values with > 50% missing data

df2 = df.copy()

df2.drop(['weight', 'payer_code', 'medical_specialty'], axis=1, inplace=True)

# Feature engineering

### Convert ages from nominal to ordinal values

In [59]:
df2.age.replace({'[0-10)':0,
              '[10-20)':1,
              '[20-30)':2,
              '[30-40)':3,
              '[40-50)':4,
              '[50-60)':5,
              '[60-70)':6,
              '[70-80)':7,
              '[80-90)':8,
               '[90-100)':9}, inplace=True)

One of the major challenges will be finding ways to reduce the number of groups in the categorical variables. According to the description, some of the coded variables have over 900 distinct nominal values. If we wanted to model these features it would be simply too much data to process. We can aggregate the large varieties of groups into smaller broader categories.

### Convert diagnosis features to broader categories using ICD-9 codes 

Some values are coded using alpha chars, these will be difficult to sort through so let's convert everything to a numeric char and then set the dtype to float before converting to broader categories.

In [60]:
# Change V01-V91 format to 1000-1099 and E000-E999 to 2000-2999, convert dtype to float

diagnoses = ['diag_1', 'diag_2', 'diag_3']

for var in diagnoses:
    
    # Convert V01-V91 format to 1000-1099, must convert to str so that the second loop doesnt raise error
    for i in list((df2.loc[df2[var].str.contains('V'), var]).index):
        df2.loc[i, var] = str(int(df2.loc[i, var][1:]) + 1000)

    # Convert  E000-E999 to 2000-2999
    for j in list((df2.loc[df2[var].str.contains('E'), var]).index):
        df2.loc[j, var] = str(int(df2.loc[j, var][1:]) + 2000)


There remains a '?' character which represents null values, we will set these values to Nan.

In [61]:
# Convert dtypes to float, set nan to 0 to avoid errors when categorizing

for var in diagnoses:
    df2.loc[:, var] = pd.to_numeric(df2.loc[:,var], errors='coerce')
    
df2.loc[:,diagnoses].fillna(0, inplace=True)

In [62]:
replace_list = ['infections, parasites', 'neoplasms', 'endocrine, nutr, metab, immun', 'blood', 'mental', 
                'nervous', 'circulatory', 'respiratory', 'digestive', 'genitourinary', 'pregnancy/childbirth',
               'skin', 'musculoskeletal', 'congenital', 'perinatal', 'ill-defined conditions', 'injury/poisioning',
               'supplementary classification of factors influencing health status',
               'supplementary classification of causes of injury/poisoning']

cutoffs = [1, 140, 240, 280, 290, 320, 390, 460, 520, 580, 630, 680, 710, 740, 760, 780, 800, 1000, 2000, 4000]

# Use the replace_list and cutoffs to sort the diagnosis variables into new features

for var in diagnoses:
    newvar = var + '_sorted'
    df2[newvar] = df2[var]
    for i in range(len(replace_list)):
        df2.loc[(df2[var] >= cutoffs[i]) & (df2[var] < cutoffs[i+1]), newvar] = replace_list[i]

In [63]:
df2[['diag_1_sorted','diag_2_sorted','diag_3_sorted']].head()

,diag_1_sorted,diag_2_sorted,diag_3_sorted
0,"endocrine, nutr, metab, immun",NaN,NaN
1,"endocrine, nutr, metab, immun","endocrine, nutr, metab, immun","endocrine, nutr, metab, immun"
2,pregnancy/childbirth,"endocrine, nutr, metab, immun",supplementary classification of factors influe...
3,"infections, parasites","endocrine, nutr, metab, immun",circulatory
4,neoplasms,neoplasms,"endocrine, nutr, metab, immun"


### Create a feature that sums the total number of hospital visits in previous year

In [64]:
df2['num_hosp_visits'] = df2['number_outpatient'] + df2['number_inpatient'] + df2['number_emergency']

### Create binary feature that categorizes discharge as 'discharged to home' or not

In [65]:
df2['discharged_home'] = np.where(df2['discharge_disposition_id'] == 1, 1, 0)

### Create binary features that categorizes admission by emergency room, referral, or other

In [66]:
df2['emergency_room'] = np.where(df2['admission_source_id'] == 7, 1, 0)
df2['referral'] = np.where((df2['admission_source_id'] == 1) | (df2['admission_source_id'] == 2) |
                           (df2['admission_source_id'] == 3), 1, 0)

### Create two outcome variables, readmittance and readmittance within 30 days

In [67]:
df2['binary_readmitted'] = np.where(df2['readmitted'] == 'NO', 0, 1)
df2['binary_readmitted30'] = np.where(df2['readmitted'] == '<30', 1, 0)

### Drop duplicate patient visits

In [68]:
df2.drop_duplicates(subset='patient_nbr', keep='last', inplace=True)

In [69]:
df2.shape

(71518, 56)

### Split into train/test sets

In [70]:
y = df2['binary_readmitted30']

# Add numeric and ordinal features

X = df2.loc[:, ('age', 'time_in_hospital', 'num_lab_procedures', 'num_procedures','num_medications',
                'number_outpatient', 'number_emergency','number_inpatient', 'number_diagnoses', 'num_hosp_visits',
               'discharged_home', 'emergency_room', 'referral')]
# Add categorical dummies

X = pd.concat([X, pd.get_dummies(df2.loc[:,('race', 'gender', 'max_glu_serum', 'A1Cresult', 'metformin', 
                                            'insulin','change','diabetesMed','diag_1_sorted')], 
                                 drop_first=True)], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.1, random_state = 2)

In [71]:
mlp = MLPClassifier(hidden_layer_sizes=(26,13), random_state=2, verbose=True, warm_start=True, 
                    early_stopping=False, alpha=.001)
mlp.fit(X_train,y_train)

Iteration 1, loss = 0.26574203
Iteration 2, loss = 0.18176394
Iteration 3, loss = 0.17787353
Iteration 4, loss = 0.17689020
Iteration 5, loss = 0.17526695
Iteration 6, loss = 0.17470273
Iteration 7, loss = 0.17408326
Iteration 8, loss = 0.17407381
Iteration 9, loss = 0.17373284
Iteration 10, loss = 0.17359273
Iteration 11, loss = 0.17270670
Iteration 12, loss = 0.17298002
Iteration 13, loss = 0.17272599
Iteration 14, loss = 0.17225259
Iteration 15, loss = 0.17228425
Iteration 16, loss = 0.17142012
Iteration 17, loss = 0.17172510
Iteration 18, loss = 0.17131600
Iteration 19, loss = 0.17143592
Iteration 20, loss = 0.17115148
Iteration 21, loss = 0.17118773
Iteration 22, loss = 0.17126165
Iteration 23, loss = 0.17124413
Iteration 24, loss = 0.17141976
Iteration 25, loss = 0.17028461
Iteration 26, loss = 0.17016413
Iteration 27, loss = 0.17034021
Iteration 28, loss = 0.16994639
Iteration 29, loss = 0.16999897
Iteration 30, loss = 0.16976203
Iteration 31, loss = 0.17053576
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(26, 13), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=2, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=True, warm_start=True)

In [72]:
mlp.score(X_test,y_test)

0.950503355704698

In [73]:
recall_score(y_test, mlp.predict(X_test))

0.011461318051575931

In [74]:
precision_score(y_test, mlp.predict(X_test))

0.3076923076923077

In [79]:
X_down = pd.concat([X_train,y_train], axis=1)

# Seperate into readmitted/not_readmitted

readmitted = X_down[X_down['binary_readmitted30'] == 1]
not_readmitted = X_down[X_down['binary_readmitted30'] == 0]

# Undersampling the majority class

not_readmitted_down = resample(not_readmitted, 
                              replace=False,
                              n_samples=len(readmitted),
                              random_state=1)

# Combine classes

X_down = pd.concat([not_readmitted_down, readmitted])

# Check class counts

X_down['binary_readmitted30'].value_counts()

# Split our upsampled data 

y_train = X_down['binary_readmitted30']
X_train = X_down.drop('binary_readmitted30', axis=1)

In [88]:
mlp = MLPClassifier(hidden_layer_sizes=(26,13), random_state=2, verbose=True, warm_start=True, 
                    early_stopping=False, alpha=.001)
mlp.fit(X_train,y_train)

Iteration 1, loss = 0.88321477
Iteration 2, loss = 0.69155602
Iteration 3, loss = 0.67767354
Iteration 4, loss = 0.67512441
Iteration 5, loss = 0.67097983
Iteration 6, loss = 0.66582739
Iteration 7, loss = 0.66581555
Iteration 8, loss = 0.65991343
Iteration 9, loss = 0.65790510
Iteration 10, loss = 0.65404656
Iteration 11, loss = 0.65394705
Iteration 12, loss = 0.65091622
Iteration 13, loss = 0.64652464
Iteration 14, loss = 0.64621738
Iteration 15, loss = 0.64245320
Iteration 16, loss = 0.64310763
Iteration 17, loss = 0.64153325
Iteration 18, loss = 0.64096727
Iteration 19, loss = 0.63885371
Iteration 20, loss = 0.63907191
Iteration 21, loss = 0.63582129
Iteration 22, loss = 0.63536799
Iteration 23, loss = 0.63409542
Iteration 24, loss = 0.63290544
Iteration 25, loss = 0.63227287
Iteration 26, loss = 0.63160510
Iteration 27, loss = 0.63058330
Iteration 28, loss = 0.63073120
Iteration 29, loss = 0.62859028
Iteration 30, loss = 0.62631874
Iteration 31, loss = 0.62546609
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(26, 13), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=2, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=True, warm_start=True)

In [89]:
mlp.score(X_test,y_test)

0.6988255033557047

In [90]:
recall_score(y_test, mlp.predict(X_test))

0.504297994269341

In [91]:
precision_score(y_test, mlp.predict(X_test))

0.08159480760315253

# Summary

The model is not performing any better than a Random Forest Classifier I created in another project. While accuracy was high, sensitivity and precision are low. Even with attempts at resampling the data the model was unable to significantly improve its performance on this problem.